The goal of this final project is to use an LLM and RAG to select the most qualified candidates (with explanation) for a given job posting given a large dataset of resumes.

#Installations

In [ ]:
!pip install langchain sentence-transformers beautifulsoup4 bert-score PyPDF2 langgraph langchain-chroma langchain-huggingface huggingface_hub kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.3/433.3 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta

#Set Up

In [ ]:
#import libraries
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
import requests
from PyPDF2 import PdfReader
from bs4 import BeautifulSoup as Soup
from bert_score import score
from huggingface_hub import InferenceClient
import os
import zipfile
import kagglehub
import pandas as pd

#hugging face api key

#kaggle api key for resume dataset

In [ ]:
#pull dataset from kaggle
!kaggle datasets download -d snehaanbhawal/resume-dataset -p data --unzip

Dataset URL: https://www.kaggle.com/datasets/snehaanbhawal/resume-dataset
License(s): CC0-1.0


#Code

##data set up

In [ ]:
#dataset path
path = kagglehub.dataset_download("snehaanbhawal/resume-dataset")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/resume-dataset


I found a dataset on kaggle that has over 2000 resumes to use as external knowledge for RAG on my LLM.

source - https://www.kaggle.com/datasets/snehaanbhawal/resume-dataset/data

Of course, there are alternative ways I could solve this, but I thought a real-life scenario would be most interesting. In practice, a job posting will have at least a 1000+ people applying, if not more. If a company can automate the hiring process to save money, resources, and time, while still capturing the most qualified candidates for the next step in the hiring process (assumably run by a human recruiter) this would be a worth while investment.

The one caveat with this approach analyzing our model's performance. In a scenario such as this, it is critical that the model has extremely few false negatives. As a company looking for the cream of the crop workers, there can not be highly qualified candidates who are slipping through the cracks because our model does not think they are qualified for the position due to poor performance. On the otherhand, false positives are ok to a certain extent. If we get a handful of false positives that is easy enough for our human recruiters to parse through in the next phase of the hiring process. So it is important to keep that in mind when using a LLM for this task.

On top of that the dataset is unsupervised. This will make it more difficult to evaulate our model. If our dataset was labeled it would be easy to just see if the results returned from the LLM are as we expect when looking at the labels (accuracy). I am going to attempt to tackle this in a couple different ways which will be explained at the top of the evaulte section of the notebook.

In [ ]:
#check to make sure the dataset is there
resumes_path = '/kaggle/input/resume-dataset/Resume/Resume.csv'
resume_df = pd.read_csv(resumes_path)
resume_df

#only testing with 10 because it takes a long time to load in thousands of resumes to my vectordatabase
#resume_df = resume_df[:10]

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION


You can see here that the dataset is 2484 rows long (2484 resumes) and the resume_str value is the a string value of an entire resume. If you look at the kaggle dataset on the website, they have the actual pdf of the resume as well. I am just going to use the csv for the project since I only need string data, but you could just as easily convert the pdf into a txt string for each resume before feeding it into a vector database for our model.

It is important to note that in the real-world it would be nice to have the pdf linked to the ID number on the csv to easily be able to look at qualified candidates pdf resumes. The format of a person's resume can be another factor used for hiring.

In [ ]:
#sanity check
resume_df['Resume_str'][0]

"         HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           Accomplishments      Missouri DOT Supervisor Training Certification  Certified by IHG in Customer Loyalty and Marketing by Segment   Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    Hilton OnQ  ,   Micros    Opera PMS   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and seminars in customer service, sales strategies, inventory control, loss pr

RAG is an important part of the project. The resumes are going to be fed to the LLM as external knowledge to help with the query. The job description at hand is going to be part of the prompt.

source - https://www.scrapingbee.com/blog/how-to-scrape-all-text-from-a-website-for-llm-ai-training/

In [ ]:
#load job description and return as txt
def load_job_desc(path):
    response = requests.get(path)

    print(response)
    print('\n')

    soup = Soup(response.text, 'html.parser')
    text = soup.get_text(separator='\n')

    #split text into list based on each line
    lines = text.splitlines()

    #get rid of white space on long lines (cleaning text)
    lines = [line.strip() for line in lines if len(line.strip()) > 40]

    #join text back
    text = "\n".join(lines)

    return text

#test job posting for notebook
job_txt = load_job_desc('https://independentsoftware.applytojob.com/apply/9eZwPPE5rn/Data-Scientist-Level-2?source=INDE&~&_jvst=JobBoard&_jvsd=Indeed')
job_txt

<Response [200]>




'Data Scientist, Level 2 - Independent Software - Career Page\nIndependent Software is a consulting, product, and solutions firm dedicated to the practical application of software and system engineering technologies to solve complex problems.\xa0\xa0We bring together world class engineers with proven engineering best practices, domain expertise, commercial technologies, and proven agile management approaches to create high value solutions aimed at helping our customers meet their most critical business and mission objectives.\nWe are focused on continual learning and evolution.\xa0\xa0We don’t do things because “that’s the way we’ve always done things”; we listen to our employees and adapt to the changing marketplace.\xa0\xa0We look at the big picture and encourage our engineers to get training and certifications in emerging technologies that will help shape our customer’s mission.\nWe’ve been profitable year after year.\xa0\xa0We’re always on the lookout for great engineers to join th

Here I went on to Indeed.com and searched for data scientist job postings. I found the job posting linked at below. It looks like a standard job posting with title, what you will be doing, description, requirements such as experience and education, clearance, location, etc. In theory this same process should work for any job posting you want.

Essentially, I just took the job posting and scraped the webpage to gather the relevant text and put it into a string to feed my LLM later on in the process as part of the query.

source - https://independentsoftware.applytojob.com/apply/9eZwPPE5rn/Data-Scientist-Level-2?source=INDE&~&_jvst=JobBoard&_jvsd=Indeed

##model/RAG part

RAG components -> load doc to text, chunk text, store in vector db

all-MiniLM-L6-v2 is 'a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.'

When it comes to job description and resume langauge semantics are important to capture correctly.

source for sentence transformer selection - https://old.reddit.com/r/LocalLLaMA/comments/16cdsv6/which_sentence_transformer_is_the_best_one_for/

Source - https://python.langchain.com/docs/tutorials/rag/

Source - https://python.langchain.com/docs/integrations/providers/huggingface/

hugging face emebdding - https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [ ]:
#load documents
docs = [Document(page_content=row['Resume_str'], metadata={"ID": str(row['ID'])}) for _, row in resume_df.iterrows()]

Storing the resume text in the vector database does take a decent chunk of time due to the large amount of text from the resume dataset.

In [ ]:
#split text
txt_split = RecursiveCharacterTextSplitter(chunk_size=1250, chunk_overlap=250)
splits = []

for doc in docs:
    chunks = txt_split.split_documents([doc])
    for c in chunks:
        #correlate ID to each resume within chunking
        c.metadata = doc.metadata
        splits.append(c)

#call embedding
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#store embeddings in vectordb
res_db = Chroma.from_documents(splits, embedding=embeddings, persist_directory="./candidate_db")
retriever = res_db.as_retriever(search_type="similarity", k=3)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

So now that we have stored the resumes in the vector database we can go on to the query part of the code. Essentially we are asking the LLM what RESUME CHUNKS (from the vector database) are most qualified to do the job on the job posting (stored in job_txt).

important to note it is not an entire resume but chunks of resumes. LLM uses retriever to get the top k similar chunks given the query (job description).

In [ ]:
# Retrieve the relevant resume text based on the job description
retrieved_docs = retriever.invoke(job_txt)

#debugging
'''for doc in retrieved_docs:
    print(doc.metadata)
'''

# Combine retrieved resume documents
resumes = []
for doc in retrieved_docs:
    res_id = doc.metadata['ID']
    resumes.append(f"Candidate ID: {res_id}\nResume:\n{doc.page_content.strip()}")

retrieved_res_txt = "\n\n".join(resumes)

# Combine resume docs and job posting into a final prompt
prompt = f"""

Based on my job posting give me the 3 most qualified candidates based on the resumes given. Identify each candidate by the resume ID value. If there are no qualified candidates that is ok. Just mention why no one qualified. Do not invent any details.

Job Description:
{job_txt}

Resumes:
{retrieved_res_txt}
"""

The model I am going to use is HuggingFaceH4/zephyr-7b-beta. It has 7B parameters.

Zephyr is a series of language models that are trained to act as helpful assistants. Zephyr-7B-β is the second model in the series, and is a fine-tuned version of mistralai/Mistral-7B-v0.1 that was trained on on a mix of publicly available, synthetic datasets using Direct Preference Optimization (DPO). We found that removing the in-built alignment of these datasets boosted performance on MT Bench and made the model more helpful. However, this means that model is likely to generate problematic text when prompted to do so. You can find more details in the technical report.

source - https://huggingface.co/HuggingFaceH4/zephyr-7b-beta

In [ ]:
#model
client = InferenceClient(model="HuggingFaceH4/zephyr-7b-beta")

#output based on prompt and RAG info
response = client.chat_completion(
    messages=[
        {"role": "system", "content": "You are an initial screening recruiter tasked with providing a list of the most qualified candidate based on the resumes present. If there is no qualified candidates that is ok."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=2000,
    temperature=0.7,
)

print(response)

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content="Based on the provided job description and resumes, the three most qualified candidates for the Data Scientist, Level 2 position at Independent Software are:\n\n1. Candidate ID: 51588273\n   - Python expertise and experience in automating workflows, data manipulation, and visualization using Jupyter Notebooks\n   - Bachelor's Degree with 10 years of relevant experience or Associate's degree with 12 years of relevant experience\n   - Clearance Required: Must possess an active TS/SCI with Full Scope Polygraph security clearance\n   - Experience in data processing, modeling, inference, and prediction\n   - Skills in mathematics, statistics, computer science, and domains such as physics, chemistry, biology, astronomy, or behavioral, social, or life sciences\n   - Proficient in Python\n\n2. Candidate ID: 11813872\n   - Experience in financia

I did want to restrict the output size of the LLM so I set the max_tokens limit to a very high number. This would allow the LLM to output a good amount of explanation for the selections to be used for evaluation. In addition, I put .7 as the temperature because that was what was used in the example on the source, and I liked the wording of the LLMs output, although for something as straightforward as my task, a lower temperature may be more suitable.

##output

In [ ]:
#output
print(response.choices[0].message.content)

Based on the provided job description and resumes, the three most qualified candidates for the Data Scientist, Level 2 position at Independent Software are:

1. Candidate ID: 51588273
   - Python expertise and experience in automating workflows, data manipulation, and visualization using Jupyter Notebooks
   - Bachelor's Degree with 10 years of relevant experience or Associate's degree with 12 years of relevant experience
   - Clearance Required: Must possess an active TS/SCI with Full Scope Polygraph security clearance
   - Experience in data processing, modeling, inference, and prediction
   - Skills in mathematics, statistics, computer science, and domains such as physics, chemistry, biology, astronomy, or behavioral, social, or life sciences
   - Proficient in Python

2. Candidate ID: 11813872
   - Experience in financial instruments modeling and portfolio and investment management
   - Expertise in Oracle Exadata, Amazon Redshift, SQL, Python, Java, Apache Ignite, AWS, and related

Based on the inital response from the model the results seem acceptable.

In [ ]:
#pd.set_option('display.max_colwidth', 50)

#candidate 1 resume txt - Category: Engineering
candidate_1 = resume_df[resume_df['ID'] == 51588273]
candidate_1_res = candidate_1['Resume_str'].values[0]
candidate_1_res

"         SOFTWARE ENGINEERING MANAGER           Summary    Multifaceted technical career with 15+ years' track record of innovation and success.\nAccomplished, enthusiastic, and driven Software Engineer with a solid history of effective systems engineering in Client/Server/ Web / Enterprise software and SaaS / SOA development. Well recognized for strong leadership and project management abilities while leading cross-functional teams in fast-paced, competitive work environments.  Willing to multitask on projects in personnel and database management and architecture/design.      Highlights        C#, ASP.NET, MVC, SQL, JavaScript, Java, Web Services, Agile, Scrum, ETL, PHP\nTools:\nVisual Studio, SSRS, SSIS, IIS, SharePoint, Eclipse, Apache, Cognos, SQL Server, Oracle              Experience      Software Engineering Manager    August 2011   to   Current     Company Name   －   City  ,   State      Orchestrate key internal IT developments for large-scale enterprise solutions while managi

In [ ]:
#candidate 2 resume txt - Category: Agriculture
candidate_2 = resume_df[resume_df['ID'] == 11813872]
candidate_2_res = candidate_2['Resume_str'].values[0]
candidate_2_res

'         VP, PRINCIPAL       Summary     I am highly skilled,growth mindset IT professional having more than 20 years experience mostly in financial industry related with providing advanced data solutions using innovative database technology. Very innovative,creative, great problem solver and have achieved the highest ratings consistently for more than 10 years. Continuously learning,adapting and evolving by overcoming challenges faced during professional career. I am fortunate to be a part of team who has delivered cutting edge products over the years to help our firm and clients. My career philosophy is  4LT(Listen,Learn,Love,Lead and earn Trust).        Skills          Deep expertise in designing,developing, implementing and running mission critical systems involving OLTP,OLAP and HTAP workloads  Extensive experience in building and deploying large scale applications in cloud environment(AWS)  Deep expertise in advanced data modeling, data management and data governance  Passionate

In [ ]:
#candidate 3 resume txt - Category: Sales
candidate_3 = resume_df[resume_df['ID'] == 29134721]
candidate_3_res = candidate_3['Resume_str'].values[0]
candidate_3_res

'         SALES       Career Overview     During the course of my career as an IT Professional, I have developed a varied set of I.T. and business related skills.  I hold expert knowledge in applying technology to business processes resulting in a more cost effective and efficient enterprise.  My accomplishments as a major player in the successful pioneering and implementation of the "Paperless Office" in 33 local Virginia agencies, created a paradigm shift in the culture of how agencies currently do business.  The experience of being a road warrior leading the day to day "hands on" training of the user afforded me a "real life" work experience and invaluable insight in user training needs, challenges, acceptance and buy-in.  The knowledge I gained from the users resulted into successful problem solving, change management and implementation outcomes of enterprise management systems. This kind of knowledge and experience cannot be gained in a classroom or from a white paper.  I have a s

In [ ]:
candidate_df = resume_df[resume_df['ID'].isin([29134721, 11813872, 51588273])]
candidate_df

,ID,Resume_str,Resume_html,Category
929,11813872,"VP, PRINCIPAL Summary I am ...","<div class=""RNA skn-cnt4 fontsize fontface vma...",AGRICULTURE
1001,29134721,SALES Career Overview Durin...,"<div class=""fontsize fontface vmargins hmargin...",SALES
1742,51588273,SOFTWARE ENGINEERING MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",ENGINEERING


Looking further at these specific resumes (without knowledge of other potential candidates that the model did not look at), they seem like strong candidates for the position. Interestingly, the category labeled on Kaggle for these 3 resumes are engineering, agriculture, and sales. I am not sure how Kaggle is going about categorizing each resume, but data science can really be applied to any industry nowadays so I am not going to put too much importance on the category of the resume.

Looking deeper into the resume text of each of the selected candidates, they seem like very suitable candidates for the job posting. Of course, this is just initial human feedback to the output. To get a better understanding of how well our model is performing we will need to use evaluation techniques beyond human feedback (although that is a good evaluation technique, but it is tedious).

##Model Evaluation

Referencing my explantion from the beginning of the code section, performance is more difficult to measure on unsupervised data. The question we want to ask is how well do the resumes from the LLM's output match up to the job posting. To do that something with semantic similarity would be a good way to measure performance.

In addition,

source - https://huggingface.co/sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import util

#same pretrained model as before for storing resume txt
model = SentenceTransformer('all-MiniLM-L6-v2')

def sem_sim(resumes, job_txt):
    resume_embeddings = model.encode(resumes.tolist())
    job_embedding = model.encode(job_txt)
    similarities = util.cos_sim(resume_embeddings, job_embedding)

    return similarities

In [ ]:
semantic_scores = sem_sim(candidate_df['Resume_str'], job_txt)

candidate_df = candidate_df.copy()
semantic_scores_cpu = semantic_scores.cpu()
semantic_scores_np = semantic_scores_cpu.numpy()
semantic_scores_list = semantic_scores_np.tolist()
candidate_df['score'] = semantic_scores_list
candidate_df = candidate_df.sort_values(by='score', ascending=False).reset_index(drop=True)

candidate_df
#semantic_scores_list.sort(reverse=True)
#len(semantic_scores_list)

,ID,Resume_str,Resume_html,Category,score
0,51588273,SOFTWARE ENGINEERING MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",ENGINEERING,[0.6475182771682739]
1,29134721,SALES Career Overview Durin...,"<div class=""fontsize fontface vmargins hmargin...",SALES,[0.4946720600128174]
2,11813872,"VP, PRINCIPAL Summary I am ...","<div class=""RNA skn-cnt4 fontsize fontface vma...",AGRICULTURE,[0.49313050508499146]


One question I had when doing the evaluation is how useful similarity score would be given that I am retrieving the documents using similarity score. From my understanding, the important distinction is that during retrieval the retriever grabs the highest scored chunks. This could be chunks from the same resume or different resumes. This can cause output from my LLM that may not match similiarity score for entire resumes. This is why I may have output from an unlikely source such as a resume from agriculture or sales. Even if the entire resume may not be a great candidate for the job, a chunk of the resume performed very well.

Then the model will take our retrieved documents (hopefully from multiple resumes) and do things other than just use a similarity score to justify the output response. For example, summarizing and reasoning could play a role in the response output.

Now that we have the response output of the LLM we can again use similarity score to evalute the output to see if the resume the LLM selected as whole scores highly.

As you can see, the resume from an engineer scored the highest which makes sense as that could be resume with very similar textual meaning to the data scientist job posting, while the other two scored decently but not close to resume 51588273. Again, it is important to note this is more than just looking at exact word match, but the semantic similarity between language in the job posting and the resume.

Again, context is everything when thinking about LLMs. For our task, lower scores could be acceptable because we want to cast a wide net to make sure no highly qualified candidate slips through the cracks. But at the end of the day, if we missed a candidate because their resume is poorly done to match our job description that is on them. It is also important not to cast too wide a net to keep it managable for a human recruiter to parse through results. So it is a balance. So while the other 2 scores are not ideal based on semantic similarity score, they may still be valuable candidates to look into further for the job posting. The LLM can be more of a baseline to filter our any candidates who are not worth the time to have a human look into.

To summarize, just because a resume has a high similarity score to the job posting does not mean they are instantly the best person for the job. It just means they have text in their resume that is similar in meaning to the job. The could have just fabriacted various things to make it look that way. And vice versa for a low score.

In [ ]:
semantic_scores = sem_sim(resume_df['Resume_str'], job_txt)

resume_df = resume_df.copy()
semantic_scores_cpu = semantic_scores.cpu()
semantic_scores_np = semantic_scores_cpu.numpy()
semantic_scores_list = semantic_scores_np.tolist()
resume_df['score'] = semantic_scores_list
resume_df = resume_df.sort_values(by='score', ascending=False).reset_index(drop=True)

In [ ]:
resume_df.head(10)

,ID,Resume_str,Resume_html,Category,score
0,51588273,SOFTWARE ENGINEERING MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",ENGINEERING,[0.6475182771682739]
1,27606527,SOFTWARE QUALITY ASSURANCE ANALYST II...,"<div class=""fontsize fontface vmargins hmargin...",BANKING,[0.5791860818862915]
2,32563518,SR. PROJECT MANAGER Profile ...,"<div class=""fontsize fontface vmargins hmargin...",HEALTHCARE,[0.5690450072288513]
3,20748929,BUSINESS DEVELOPMENT Career Ove...,"<div class=""fontsize fontface vmargins hmargin...",BUSINESS-DEVELOPMENT,[0.5680555701255798]
4,37335325,ENGINEERING MANAGER Career ...,"<div class=""fontsize fontface vmargins hmargin...",ENGINEERING,[0.5673145055770874]
5,12011623,ENGINEERING AND QUALITY TECHNICIAN ...,"<div class=""fontsize fontface vmargins hmargin...",ENGINEERING,[0.5640102624893188]
6,28630325,ENGINEERING TEAM LEAD Career Ov...,"<div class=""fontsize fontface vmargins hmargin...",ENGINEERING,[0.5602080225944519]
7,37242217,INFORMATION TECHNOLOGY CONSULTANT ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,[0.5592827796936035]
8,20314980,CONSULTANT Summary I am an e...,"<div class=""fontsize fontface vmargins hmargin...",CONSULTANT,[0.557654857635498]
9,20001721,INFORMATION TECHNOLOGY STUDENT ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY,[0.5569890141487122]


Here are the top scoring resumes. Interestingly the LLM did capture the #1 resume (which is #1 by a large margin). The scores here are more holistic than the vector database used in retreival to LLM. In addition, the LLM uses a more 'human' (reasoning) method to choose the most qualified candidates instead of cosine similarity so that could impact result causing differences. Even still, the other 2 resumes output by the LLM are almost close enough to be relatiev to the top 10. They are only about 5% off or so.

Based on the evaluation metric and the model output resume 51588273 is a very strong candidate.